# SQL Azure Database Basic Monitor - Serverless Aggregation Queries

Basic monitoring queries to get information about your SQL Azure Database performance.

## Prerequisites

1. An Azure Synapse Workspace with a Synapse SQL Serverless Pool
2. Permission provided to the Serverless Pool Managed Identity to access the storage account
3. Data feeding into the storage account

## Initial Setup
These scripts are run to set up your initial views/access to the CSV files. These assume you have the proper permissions set up from your serverless pool to access your blob storage account.

### Credentials
These are the credentials and data source to access the CSV files, make sure to change the location to your storage account and container.
Comment these lines out if you have already created your sql credentials to access the data.

In [ ]:
CREATE MASTER KEY;

CREATE DATABASE SCOPED CREDENTIAL SynapseId
WITH IDENTITY = 'Managed Identity';

CREATE EXTERNAL DATA SOURCE SqlLogging
WITH (    LOCATION   = 'https://[youraccount].blob.core.windows.net/[yourcontainer]/',
          CREDENTIAL = SynapseId
);

### Performance View
This view will give you access to the Sql Performance Data retained from `sys.dm_db_resource_stats`

Make sure to edit the FIELDTERMINATOR to what you used as your LOG_FILE_CSV_SEPARATOR in your function settings.

In [29]:
CREATE OR ALTER VIEW SqlAzurePerformance
AS
SELECT CAST(end_time as DATETIME) as end_time, [avg_cpu_percent], [avg_data_io_percent], [avg_log_write_percent], [avg_memory_usage_percent], [xtp_storage_percent], [max_worker_percent], [max_session_percent], [dtu_limit], [avg_login_rate_percent], [avg_instance_cpu_percent], [avg_instance_memory_percent], [cpu_limit], [replica_role], CAST(perf.filepath(1) as DATE) as [file_date] FROM
    OPENROWSET(
        BULK 'performance-*.csv',
        FORMAT = 'CSV', 
        PARSER_VERSION = '2.0',
        FIELDTERMINATOR ='|',
        DATA_SOURCE = 'SqlLogging',
        ROWSET_OPTIONS = '{"READ_OPTIONS":["ALLOW_INCONSISTENT_READS"]}'
    )
    WITH (
        [end_time] VARCHAR(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [avg_cpu_percent] DECIMAL(8,2),
        [avg_data_io_percent] DECIMAL(8,2),
        [avg_log_write_percent] DECIMAL(8,2),
        [avg_memory_usage_percent] DECIMAL(8,2),
        [xtp_storage_percent] DECIMAL(8,2),
        [max_worker_percent] DECIMAL(8,2),
        [max_session_percent] DECIMAL(8,2),
        [dtu_limit] INT,
        [avg_login_rate_percent] DECIMAL(8,2),
        [avg_instance_cpu_percent] DECIMAL(8,2),
        [avg_instance_memory_percent] DECIMAL(8,2),
        [cpu_limit] INT,
        [replica_role] INT
    ) 
    as [perf]

Commands completed successfully.

Total execution time: 00:00:08.019

### Extended Performance Views

If you enabled the extended performance captures, you can use these views to access your extended performance and wait stat information.

Like above, make sure to edit the FIELDTERMINATOR to what you used as your LOG_FILE_CSV_SEPARATOR in your function settings.


In [18]:
CREATE OR ALTER VIEW SqlAzureExtendedPerformance
AS
SELECT 
        CAST([reading_time] as DATETIME) as [reading_time],
        [session_id],
        [request_id],
        [ecid],
        [blocking_session_id],
        [blocking_ecid],
        [task_state],
        [wait_type],
        [wait_duration_ms],
        [wait_resource],
        [resource_description],
        [last_wait_type],
        [open_trans],
        [transaction_isolation_level],
        [is_user_process],
        [request_cpu_time],
        [request_logical_reads],
        [request_reads],
        [request_writes],
        [memory_usage],
        [session_cpu_time],
        [session_reads],
        [session_writes],
        [session_logical_reads],
        [total_scheduled_time],
        [total_elapsed_time],
        CAST([last_request_start_time] as DATETIME) as [last_request_start_time],
        CAST([last_request_end_time] as DATETIME) as [last_request_end_time],
        [session_row_count],
        [prev_error],
        [open_resultsets],
        [request_total_elapsed_time],
        [percent_complete],
        [est_completion_time],
        [tran_name],
        CAST([transaction_begin_time] as DATETIME) as [transaction_begin_time],
        [tran_type],
        [tran_state],
        CAST([request_start_time] as DATETIME) as [request_start_time],
        [request_status],
        [command],
        CONVERT(varbinary(64), [plan_handle], 1) as [plan_handle],
		CONVERT(varbinary(64), [sql_handle], 1) as [sql_handle],
        [statement_start_offset],
        [statement_end_offset],
        [database_id],
        [user_id],
        [executing_managed_code],
        [pending_io_count],
        CAST([login_time] as DATETIME) as [login_time],
        [host_name],
        [program_name],
        [host_process_id],
        [client_version],
        [client_interface_name],
        [login_name],
        [nt_domain],
        [nt_user_name],
        [net_packet_size],
        [client_net_address],
        CONVERT(varbinary(64), [most_recent_sql_handle], 1) as [most_recent_sql_handle],
        [session_status],
        [scheduler_id],
        [group_id],
		CONVERT(varbinary(128), [context_info], 1) as [context_info],
        CAST(perf.filepath(1) as DATE) as [file_date] FROM
    OPENROWSET(
        BULK 'extended-performance-*.csv',
        FORMAT = 'CSV', 
        PARSER_VERSION = '2.0',
        FIELDTERMINATOR ='|',
        DATA_SOURCE = 'SqlLogging',
        ROWSET_OPTIONS = '{"READ_OPTIONS":["ALLOW_INCONSISTENT_READS"]}'
    )
    WITH (
        [reading_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [session_id] [smallint],
        [request_id] [int],
        [ecid] [int],
        [blocking_session_id] [smallint],
        [blocking_ecid] [int] ,
        [task_state] nvarchar(15) COLLATE Latin1_General_100_BIN2_UTF8,
        [wait_type] [nvarchar](60) COLLATE Latin1_General_100_BIN2_UTF8,
        [wait_duration_ms] [bigint],
        [wait_resource] [nvarchar](40) COLLATE Latin1_General_100_BIN2_UTF8,
        [resource_description] [nvarchar](140) COLLATE Latin1_General_100_BIN2_UTF8,
        [last_wait_type] [nvarchar](50) COLLATE Latin1_General_100_BIN2_UTF8,
        [open_trans] [int],
        [transaction_isolation_level] [varchar](30) COLLATE Latin1_General_100_BIN2_UTF8,
        [is_user_process] [bit],
        [request_cpu_time] [int],
        [request_logical_reads] [bigint],
        [request_reads] [bigint],
        [request_writes] [bigint],
        [memory_usage] [int],
        [session_cpu_time] [int],
        [session_reads] [bigint],
        [session_writes] [bigint],
        [session_logical_reads] [bigint],
        [total_scheduled_time] [int],
        [total_elapsed_time] [int],
        [last_request_start_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [last_request_end_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [session_row_count] [bigint],
        [prev_error] [int],
        [open_resultsets] [int],
        [request_total_elapsed_time] [int],
        [percent_complete] [decimal](5, 2),
        [est_completion_time] [bigint],
        [tran_name] [nvarchar](24),
        [transaction_begin_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [tran_type] [varchar](15) COLLATE Latin1_General_100_BIN2_UTF8,
        [tran_state] [varchar](15) COLLATE Latin1_General_100_BIN2_UTF8,
        [request_start_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [request_status] [nvarchar](15) COLLATE Latin1_General_100_BIN2_UTF8,
        [command] [nvarchar](16) COLLATE Latin1_General_100_BIN2_UTF8,
        [plan_handle] [varchar](256) COLLATE Latin1_General_100_BIN2_UTF8,
        [sql_handle] [varchar](256) COLLATE Latin1_General_100_BIN2_UTF8,
        [statement_start_offset] [int],
        [statement_end_offset] [int],
        [database_id] [smallint],
        [user_id] [int],
        [executing_managed_code] [bit],
        [pending_io_count] [int],
        [login_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [host_name] [nvarchar](20) COLLATE Latin1_General_100_BIN2_UTF8,
        [program_name] [nvarchar](50) COLLATE Latin1_General_100_BIN2_UTF8,
        [host_process_id] [int],
        [client_version] [int],
        [client_interface_name] [nvarchar](30) COLLATE Latin1_General_100_BIN2_UTF8,
        [login_name] [nvarchar](30) COLLATE Latin1_General_100_BIN2_UTF8,
        [nt_domain] [nvarchar](30) COLLATE Latin1_General_100_BIN2_UTF8,
        [nt_user_name] [nvarchar](20) COLLATE Latin1_General_100_BIN2_UTF8,
        [net_packet_size] [int],
        [client_net_address] [varchar](20) COLLATE Latin1_General_100_BIN2_UTF8,
        [most_recent_sql_handle] [varchar](256)  COLLATE Latin1_General_100_BIN2_UTF8,
        [session_status] [nvarchar](15) COLLATE Latin1_General_100_BIN2_UTF8,
        [scheduler_id] [int],
        [group_id] [int],
        [context_info] varchar(512) COLLATE Latin1_General_100_BIN2_UTF8
    ) 
    as [perf]

Commands completed successfully.

Total execution time: 00:00:07.623

In [ ]:
CREATE OR ALTER VIEW SqlWaitStats
AS
SELECT CAST(reading_time as DATETIME) as reading_time, 
	[wait_type], 
	[waiting_tasks_count], 
	[wait_time_ms], 
	[max_wait_time_ms], 
	[signal_wait_time_ms], 
	CAST(perf.filepath(1) as DATE) as [file_date] FROM
    OPENROWSET(
        BULK 'wait_stats-*.csv',
        FORMAT = 'CSV', 
        PARSER_VERSION = '2.0',
        FIELDTERMINATOR ='|',
        DATA_SOURCE = 'SqlLogging',
        ROWSET_OPTIONS = '{"READ_OPTIONS":["ALLOW_INCONSISTENT_READS"]}'
    )
    WITH (
        [reading_time] VARCHAR(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [wait_type] NVARCHAR(45),
        [waiting_tasks_count] bigint,
		[wait_time_ms] bigint,
		[max_wait_time_ms] bigint,
		[signal_wait_time_ms] bigint
    ) 
    as [perf]

## Example Queries
Here are some example queries to start using the performance data. 

When writing your own queries, make sure to timebox your data using `file_date` as the predicate as this will use partition elimination to improve your query performance.

### Top 5 instances of CPU Usage over the last week

In [41]:
SELECT TOP 5 end_time, avg_cpu_percent, avg_data_io_percent, avg_log_write_percent, file_date
FROM SqlAzurePerformance WHERE file_date > DATEADD(day, -7, CURRENT_TIMESTAMP) ORDER BY avg_cpu_percent DESC

Statement ID: {8F495E70-A382-4162-A3F8-972B162F3AB8} | Query hash: 0x5EFBE0F2CC9805DD | Distributed request ID: {07AEC349-1F47-4D97-A07C-5219469200F9}. Total size of data scanned is 3 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(5 rows affected)

Total execution time: 00:00:20.277

end_time,avg_cpu_percent,avg_data_io_percent,avg_log_write_percent,file_date
2022-11-08 20:21:04.000,12.95,0.62,0.00,2022-11-08
2022-11-08 03:46:09.000,2.82,0.00,0.00,2022-11-08
2022-11-08 04:15:14.000,2.04,0.00,0.00,2022-11-08
2022-11-08 03:47:09.000,1.88,0.00,0.00,2022-11-08
2022-11-07 22:07:09.000,1.88,0.00,0.00,2022-11-07


### Extended query statistics

Based on the results of the query above we see that at 20:21 we have a higher use of CPU. With the extended performance data we can see if anything interesting stands out.

Note the reading_time being filtered to the minutes around the end_time shown, also we filter on file_date to only read the file from one day.

In [42]:
	SELECT TOP 10000 reading_time, 
      session_id, request_id, ecid, blocking_session_id, blocking_ecid, task_state, 
      wait_type, wait_duration_ms, wait_resource, resource_description, last_wait_type, 
      open_trans, transaction_isolation_level, is_user_process, 
      request_cpu_time, request_logical_reads, request_reads, request_writes, memory_usage, 
      session_cpu_time, session_reads, session_writes, session_logical_reads, total_scheduled_time, 
      total_elapsed_time, CONVERT (varchar, last_request_start_time, 126) AS last_request_start_time, 
      CONVERT (varchar, last_request_end_time, 126) AS last_request_end_time, session_row_count, 
      prev_error, open_resultsets, request_total_elapsed_time, percent_complete, 
      est_completion_time, tran_name, 
      CONVERT (varchar, transaction_begin_time, 126) AS transaction_begin_time, tran_type, 
      tran_state, CONVERT (varchar, request_start_time, 126) AS request_start_time, request_status, 
      command, statement_start_offset, statement_end_offset, database_id, [user_id], 
      executing_managed_code, pending_io_count, CONVERT (varchar, login_time, 126) AS login_time, 
      [host_name], program_name, host_process_id, client_version, client_interface_name, login_name, 
      nt_domain, nt_user_name, net_packet_size, client_net_address, session_status, 
      scheduler_id,
      group_id,
      [context_info]
    FROM SqlAzureExtendedPerformance r
    WHERE 
		(reading_time BETWEEN '2022-11-08 20:20:04.000' AND '2022-11-08 20:22:04.000' and file_date = '2022-11-08') AND
      /* One EC can have multiple waits in sys.dm_os_waiting_tasks (e.g. parent thread waiting on multiple children, for example 
      ** for parallel create index; or mem grant waits for RES_SEM_FOR_QRY_COMPILE).  This will result in the same EC being listed 
      ** multiple times in the request table, which is counterintuitive for most people.  Instead of showing all wait relationships, 
      ** for each EC we will report the wait relationship that has the longest wait time.  (If there are multiple relationships with 
      ** the same wait time, blocker spid/ecid is used to choose one of them.)  If it were not for , we would do this 
      ** exclusion in the previous query to avoid storing data that will ultimately be filtered out. */
      NOT EXISTS 
        (SELECT * FROM SqlAzureExtendedPerformance r2 
         WHERE r.session_id = r2.session_id AND r.request_id = r2.request_id AND r.ecid = r2.ecid AND r.wait_type = r2.wait_type 
           AND (r2.wait_duration_ms > r.wait_duration_ms OR (r2.wait_duration_ms = r.wait_duration_ms)))

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

Statement ID: {209DB211-8011-4DAF-AB08-EA9BF8C2363D} | Query hash: 0xF0B92662D7833AE5 | Distributed request ID: {4A34D890-8168-4422-924C-5B36074A7035}. Total size of data scanned is 11 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(7 rows affected)

Total execution time: 00:00:21.539

reading_time,session_id,request_id,ecid,blocking_session_id,blocking_ecid,task_state,wait_type,wait_duration_ms,wait_resource,resource_description,last_wait_type,open_trans,transaction_isolation_level,is_user_process,request_cpu_time,request_logical_reads,request_reads,request_writes,memory_usage,session_cpu_time,session_reads,session_writes,session_logical_reads,total_scheduled_time,total_elapsed_time,last_request_start_time,last_request_end_time,session_row_count,prev_error,open_resultsets,request_total_elapsed_time,percent_complete,est_completion_time,tran_name,transaction_begin_time,tran_type,tran_state,request_start_time,request_status,command,statement_start_offset,statement_end_offset,database_id,user_id,executing_managed_code,pending_io_count,login_time,host_name,program_name,host_process_id,client_version,client_interface_name,login_name,nt_domain,nt_user_name,net_packet_size,client_net_address,session_status,scheduler_id,group_id,context_info
2022-11-08 20:21:00.000,52,0,NULL,0,NULL,NULL,XE_LIVE_TARGET_TVF,1095,NULL,NULL,XE_LIVE_TARGET_TVF,NULL,2-Read Committed,1,541,95,24,0,0,0,0,0,0,0,0,2022-10-21T22:07:17,2022-10-21T22:07:17,0,0,1,1548846716,0.00,0,SELECT,2022-10-21T22:07:17,2-Read only,2-Active,2022-10-21T22:07:17,suspended,SELECT,46,174,1,1,0,NULL,2022-10-21T22:07:17,DB2,TdService,12168,7,Framework Microsoft SqlClient,NT AUTHORITY\SYSTEM,NT AUTHORITY,SYSTEM,8000,<named pipe>,running,NULL,2,0x
2022-11-08 20:21:00.000,53,0,NULL,0,NULL,NULL,XE_LIVE_TARGET_TVF,59095,NULL,NULL,XE_LIVE_TARGET_TVF,NULL,2-Read Committed,1,693,0,0,0,0,0,0,0,0,0,0,2022-10-21T22:07:17,2022-10-21T22:07:17,0,0,1,1548846404,0.00,0,SELECT,2022-10-21T22:07:17,2-Read only,2-Active,2022-10-21T22:07:17,suspended,SELECT,46,174,1,1,0,NULL,2022-10-21T22:07:17,DB2,TdService,12168,7,Framework Microsoft SqlClient,NT AUTHORITY\SYSTEM,NT AUTHORITY,SYSTEM,8000,<named pipe>,running,NULL,2,0x
2022-11-08 20:21:00.000,56,0,NULL,0,NULL,NULL,NULL,0,NULL,NULL,MEMORY_ALLOCATION_EXT,NULL,2-Read Committed,1,293,17,0,0,4,0,24,0,53,3,165,2022-11-08T20:20:59,2022-11-08T20:20:59,1,0,1,986,0.00,0,SELECT,2022-11-08T20:20:59,2-Read only,2-Active,2022-11-08T20:20:59,runnable,SELECT,0,280,5,1,0,NULL,2022-11-08T20:20:59,10-30-2-220,.Net SqlClient Data Provider,8032,7,.Net SqlClient Data Provider,cabattagsa,NULL,NULL,8058,52.143.81.61,running,NULL,2000000039,0x440034003600320042004100340043002D0038003000350032002D0034004300350032002D0038004600370044002D003200420033003300300046003700310036004100360036
2022-11-08 20:21:00.000,57,0,NULL,0,NULL,NULL,NULL,0,NULL,NULL,RESERVED_MEMORY_ALLOCATION_EXT,NULL,2-Read Committed,1,14,92,0,0,0,0,0,0,0,0,0,2022-11-08T20:21:00,2022-11-08T20:21:00,0,0,1,236,0.00,0,SELECT,2022-11-08T20:21:00,2-Read only,2-Active,2022-11-08T20:21:00,running,SELECT,104,11418,5,1,0,NULL,2022-11-08T20:21:00,10-30-10-192,Core Microsoft SqlClient Data Provider,2812,7,Core Microsoft SqlClient Data,cabattagsa,NULL,NULL,8058,52.154.242.237,running,NULL,2000000039,0x
2022-11-08 20:22:00.000,52,0,NULL,0,NULL,NULL,XE_LIVE_TARGET_TVF,746,NULL,NULL,XE_LIVE_TARGET_TVF,NULL,2-Read Committed,1,541,95,24,0,0,0,0,0,0,0,0,2022-10-21T22:07:17,2022-10-21T22:07:17,0,0,1,1548906382,0.00,0,SELECT,2022-10-21T22:07:17,2-Read only,2-Active,2022-10-21T22:07:17,suspended,SELECT,46,174,1,1,0,NULL,2022-10-21T22:07:17,DB2,TdService,12168,7,Framework Microsoft SqlClient,NT AUTHORITY\SYSTEM,NT AUTHORITY,SYSTEM,8000,<named pipe>,running,NULL,2,0x
2022-11-08 20:22:00.000,53,0,NULL,0,NULL,NULL,XE_LIVE_TARGET_TVF,58761,NULL,NULL,XE_LIVE_TARGET_TVF,NULL,2-Read Committed,1,693,0,0,0,0,0,0,0,0,0,0,2022-10-21T22:07:17,2022-10-21T22:07:17,0,0,1,1548906070,0.00,0,SELECT,2022-10-21T22:07:17,2-Read only,2-Active,2022-10-21T22:07:17,suspended,SELECT,46,174,1,1,0,NULL,2022-10-21T22:07:17,DB2,TdService,12168,7,Framework Microsoft SqlClient,NT AUTHORITY\SYSTEM,NT AUTHORITY,SYSTEM,8000,<named pipe>,running,NULL,2,0x
2022-11-08 20:22:00.000,57,0,NULL,0,NULL,NULL,NULL,0,NULL,NULL,RESERVED_MEMORY_ALLOCATION_EXT,NULL,

### Wait Statistics
With that data from above we see there is a query running at 20:21 that seems to use a little more CPU. Let's see what our waits look like during that timeframe. 

In [44]:
WITH waits AS
(
	SELECT *,
		waiting_tasks_count - LAG(waiting_tasks_count) OVER(PARTITION BY wait_type order by reading_time) as waiting_tasks_change,
		wait_time_ms - LAG(wait_time_ms) OVER(PARTITION BY wait_type order by reading_time) as wait_time_change
	FROM SqlWaitStats WHERE 
	(reading_time BETWEEN '2022-11-08 20:19:00.000' AND '2022-11-08 20:24:04.000' and file_date = '2022-11-08')
)
SELECT * FROM waits
WHERE waiting_tasks_change IS NOT NULL
ORDER BY reading_time, wait_time_change

Statement ID: {5421D131-23C7-4268-BDD4-92FDD056B437} | Query hash: 0x65C98D63A803F29 | Distributed request ID: {08D47A18-E288-4C9A-8775-3A93C00B45CA}. Total size of data scanned is 1 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(68 rows affected)

Total execution time: 00:00:11.607

reading_time,wait_type,waiting_tasks_count,wait_time_ms,max_wait_time_ms,signal_wait_time_ms,file_date,waiting_tasks_change,wait_time_change
2022-11-08 20:20:00.000,ASYNC_NETWORK_IO,78,55,14,15,2022-11-08,0,0
2022-11-08 20:20:00.000,PREEMPTIVE_XHTTP,54,711,73,0,2022-11-08,0,0
2022-11-08 20:20:00.000,PREEMPTIVE_OS_CRYPTOPS,36,63,3,0,2022-11-08,0,0
2022-11-08 20:20:00.000,RESERVED_MEMORY_ALLOCATION_EXT,65421,46,0,0,2022-11-08,17,0
2022-11-08 20:20:00.000,WRITELOG,1221,17394,198,169,2022-11-08,0,0
2022-11-08 20:20:00.000,MEMORY_ALLOCATION_EXT,997566,603,2,0,2022-11-08,585,0
2022-11-08 20:20:00.000,PREEMPTIVE_OS_CRYPTACQUIRECONTEXT,126,39,4,0,2022-11-08,0,0
2022-11-08 20:20:00.000,PREEMPTIVE_OLEDBOPS,36,17,15,0,2022-11-08,0,0
2022-11-08 20:20:00.000,PAGEIOLATCH_EX,1,124,124,0,2022-11-08,0,0
2022-11-08 20:20:00.000,SOS_SCHEDULER_YIELD,5872,18347,56,18346,2022-11-08,4,0


### Wait Charts
With Azure Data Studio and this data set we can also do some interesting things around charting wait times over timeframes. Such as out first example query which shows SOS_SCHEDULER_YIELD waits over time and if there are spikes along that time frame. You can see there is a query that runs approximately every 15 minutes that has to wait 100 ms for CPU. 

In [48]:
WITH waits AS
(
	SELECT reading_time, 
		wait_time_ms - LAG(wait_time_ms) OVER(PARTITION BY wait_type order by reading_time) as wait_time_change
	FROM SqlWaitStats WHERE 
	(reading_time BETWEEN '2022-11-08 17:00:04.000' AND '2022-11-08 20:00:04.000' and file_date = '2022-11-08')
    and wait_type = 'SOS_SCHEDULER_YIELD'
)
SELECT * FROM waits
WHERE wait_time_change IS NOT NULL
ORDER BY reading_time, wait_time_change

Statement ID: {3E671E82-6387-4A1A-B928-4EF2C51F05D3} | Query hash: 0x86D411F69B6BA45 | Distributed request ID: {C2713888-C0E5-479C-9130-2EFCDA553222}. Total size of data scanned is 2 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(479 rows affected)

Total execution time: 00:00:16.120

reading_time,wait_time_change
2022-11-08 12:02:00.000,0
2022-11-08 12:03:00.000,0
2022-11-08 12:04:00.000,0
2022-11-08 12:05:00.000,0
2022-11-08 12:06:00.000,0
2022-11-08 12:07:00.000,0
2022-11-08 12:08:00.000,0
2022-11-08 12:09:00.000,0
2022-11-08 12:10:00.000,100
2022-11-08 12:11:00.000,44


With this second query you can get a proportion of waits within our database. You can see in this situation we mostly wait for CPU, T-Log, and reading from disk.

In [49]:
select wait_type, wait_time_ms from SqlWaitStats WHERE reading_time in (SELECT MAX(reading_time) from SqlWaitStats) 

Statement ID: {C1A1AE0A-571E-4822-83A7-3982664CD3ED} | Query hash: 0x616C46EB9CDF4DF6 | Distributed request ID: {B247B628-1947-4786-AD6F-0524C2FDD5E2}. Total size of data scanned is 3 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(14 rows affected)

Total execution time: 00:00:16.811

wait_type,wait_time_ms
SOS_SCHEDULER_YIELD,23719
WRITELOG,17663
PAGEIOLATCH_SH,11385
MEMORY_ALLOCATION_EXT,726
PREEMPTIVE_XHTTP,711
PAGEIOLATCH_EX,124
WAIT_ON_SYNC_STATISTICS_REFRESH,122
PREEMPTIVE_OS_CRYPTOPS,63
ASYNC_NETWORK_IO,55
RESERVED_MEMORY_ALLOCATION_EXT,50
